In [1]:
# Various torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F

# torchvision
from torchvision import datasets, transforms

# ------------------------
# get up one directory 
import sys, os
sys.path.append(os.path.abspath('../'))
# ------------------------

# custom packages
import matplotlib.pyplot as plt
import models.aux_funs as maf
import optimizers as op
import regularizers as reg
import train
import math
import utils.configuration as cf
import utils.datasets as ud
from models.mnist_conv import mnist_conv

# Fix the random seed

In [2]:
random_seed = 2
cf.seed_torch(random_seed)

# Configure the expirement

In [6]:
weights_path = '../checkpoints/mnist_conv_sparse_linbreg.pth'

conf_args = {#
    # data specification
    'data_file':"../../Data",'train_split':0.95, 'data_set':"Fashion-MNIST", 'download':True,
    # cuda
    'use_cuda':False, 'num_workers':0, 'cuda_device':0, 'pin_memory':True,
    #
    'epochs':100,
    # optimizer
    'delta':1.0, 'lr':0.07, 'lamda_0':0.4, 'lamda_1':0.4, 'optim':"AdaBreg", 'conv_group':True,
    'beta':0.0,
    # initialization
    'sparse_init':0.01, 'r':[1,1,1],
    # misc
    'random_seed':random_seed, 'eval_acc':True,
}

conf = cf.Conf(**conf_args)

# Initiate model

In [9]:
model_kwargs = {'mean':conf.data_set_mean, 'std':conf.data_set_std}    

model = mnist_conv(**model_kwargs)

# Load model weights

In [10]:
model_weights = torch.load(weights_path)

# Apply the weights
model.load_state_dict(model_weights)
model = model.to(conf.device)

# Dataset

In [13]:
train_loader, valid_loader, test_loader = ud.get_data_set(conf)

# Show the model works

In [14]:
model.eval()

train.test(conf, model, test_loader)

--------------------------------------------------
Test Accuracy: 0.8602


{'acc': 0.8602, 'loss': 3.965606987476349}

# Attack the Model